In [ ]:
!pip install --verbose --no-cache-dir torch-scatter
!pip install --verbose --no-cache-dir torch-sparse
!pip install --verbose --no-cache-dir torch-cluster
!pip install torch-geometric
!pip install tensorboardX

In [1]:
import os.path as osp
import argparse

import torch
import torch.nn.functional as F
from torch_geometric.datasets import Planetoid
import torch_geometric.transforms as T
from torch_geometric.nn import GCNConv
from tqdm import tqdm
from IPython.core.debugger import set_trace

In [2]:
def edge_pred_prep(data):
    E = data.edge_index.T.tolist()
    size = len(set([n for e in E for n in e])) 
    adjacency = [[0]*size for _ in range(size)]
    for sink, source in E:
        adjacency[sink][source] = 1

    yy = []
    for i in range(len(data.x)):
        for j in range(i, len(data.x)):
              yy.append(adjacency[i][j])

    y_edge = torch.Tensor(yy).int()
    data.y = y_edge.long()
    
    mask = [False for _ in range(len(y_edge))]
    train_mask = mask.copy()
    val_mask = mask.copy()
    test_mask = mask.copy()
    
    for i in range(int(len(y_edge)*1/6)):
        train_mask[i] = True

    for i in range(int(len(y_edge)*1/6), int(len(y_edge)*2/3)):
        val_mask[i] = True

    for i in range(int(len(y_edge)*2/3), len(y_edge)):
        test_mask[i] = True
        
    data.train_mask = torch.Tensor(train_mask).bool()
    data.val_mask = torch.Tensor(val_mask).bool()
    data.test_mask = torch.Tensor(test_mask).bool()

In [15]:
class Net(torch.nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = GCNConv(35, 16, cached=True,
                             normalize=True)
        self.conv2 = GCNConv(16, 2, cached=True,
                             normalize=True)

    def forward(self):
        
        def edge_embedding(x):
            index = 0
            edge_embeddings = torch.empty([595, 2], device=torch.device('cuda:0'))
            for i in tqdm(range(len(x))):
                for j in range(i, len(x)):
                    edge_embeddings[index] = torch.mul(x[i], x[j])
                    index += 1
            return edge_embeddings

        x, edge_index, edge_weight = data.x, data.edge_index, data.edge_attr
        x = F.relu(self.conv1(x, edge_index, None))
        x = F.dropout(x, training=self.training)
        x = self.conv2(x, edge_index, None) 
        edge_x = edge_embedding(x)
        return F.log_softmax(edge_x, dim=1)

def train():
    model.train()
    optimizer.zero_grad()
    F.nll_loss(model()[data.train_mask], data.y[data.train_mask]).backward()
    optimizer.step()
    return model()


@torch.no_grad()
def test():
    model.eval()
    logits, accs = model(), []
    for _, mask in data('train_mask', 'val_mask', 'test_mask'):
        pred = logits[mask].max(1)[1]
        acc = pred.eq(data.y[mask]).sum().item() / mask.sum().item()
        accs.append(acc)
    return [accs, pred]

# Model

In [5]:
from torch_geometric.datasets import KarateClub

In [4]:
torch.set_printoptions(profile="full")

In [6]:
dataset = KarateClub()
data = dataset[0]

In [7]:
y = torch.tensor([[item] for item in data.y.tolist()])

In [8]:
new_x = torch.cat((y, data.x), 1)

In [39]:
#dataset = Planetoid(root='/tmp/Cora', name='Cora')
#data = dataset[0]

In [9]:
edge_pred_prep(data)

In [10]:
data.x = new_x

In [11]:
data

Data(edge_index=[2, 156], test_mask=[595], train_mask=[595], val_mask=[595], x=[34, 35], y=[595])

In [19]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model, data = Net().to(device), data.to(device)
optimizer = torch.optim.Adam([
    dict(params=model.conv1.parameters(), weight_decay=5e-4),
    dict(params=model.conv2.parameters(), weight_decay=0)
], lr=0.01)  # Only perform weight-decay on first convolution.



In [20]:
output = 0
best_val_acc = test_acc = 0
for epoch in range(1, 201):
    A = train()
    train_acc, val_acc, tmp_test_acc = test()[0]
    output = test()[1]
#    if val_acc > best_val_acc:
#        best_val_acc = val_acc
    test_acc = tmp_test_acc
    log = 'Epoch: {:03d}, Train: {:.4f}, Test: {:.4f}'
    print(log.format(epoch, train_acc, test_acc))

100%|█████████████████████████████████████████████████████████████████████████████████| 34/34 [00:00<00:00, 871.75it/s]

Epoch: 001, Train: 0.6263, Test: 0.7286



  0%|                                                                                           | 0/34 [00:00<?, ?it/s]

Epoch: 002, Train: 0.6768, Test: 0.8241


  0%|                                                                                           | 0/34 [00:00<?, ?it/s]

...

Epoch: 198, Train: 0.8081, Test: 0.8141


100%|█████████████████████████████████████████████████████████████████████████████████| 34/34 [00:00<00:00, 641.52it/s]


Epoch: 199, Train: 0.8081, Test: 0.8141


100%|████████████████████████████████████████████████████████████████████████████████| 34/34 [00:00<00:00, 1172.26it/s]

Epoch: 200, Train: 0.8081, Test: 0.8141
